In [6]:
import os
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

In [7]:
# Load of all the .csv files containing extracted image features
data_USG = pd.read_csv("Features/Ultrasound features.csv")
data_MMG = pd.read_csv("Features/Mammogram features.csv")
data_multimodal = pd.read_csv("Features/multimodal features.csv")
print('Done!')

Done!


In [8]:
#For one hot encoding the labels this tab is required
class_mapping = {
    'B': 0,
    'M': 1,
}
num_classes = 2

label_encoder = LabelEncoder()
data_USG['Class'] = label_encoder.fit_transform(data_USG['Class'])
data_MMG['Class'] = label_encoder.fit_transform(data_MMG['Class'])
data_multimodal['Class'] = label_encoder.fit_transform(data_multimodal['Class'])
print('Done!')

Done!


In [28]:
class AttentionBlock(nn.Module):
    def __init__(self, input_dim, output_dim, num_heads=2):
        super(AttentionBlock, self).__init__()
        
        # Self-attention mechanism
        self.multihead_attention = nn.MultiheadAttention(input_dim, num_heads)
        
        # Feedforward neural network
        self.feedforward = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.ReLU(),
            nn.Linear(512, output_dim)
        )
        
        # Layer normalization
        self.layer_norm1 = nn.LayerNorm(input_dim)
        self.layer_norm2 = nn.LayerNorm(input_dim)
        
    def forward(self, x):
        # Apply layer normalization
        x_norm = self.layer_norm1(x)
        
        # Self-attention
        attention_output, _ = self.multihead_attention(x_norm, x_norm,x_norm)
        
        # Residual connection and layer normalization
        x_residual = x + attention_output
        x_norm2 = self.layer_norm2(x_residual)
        
        # Feedforward network
        output = self.feedforward(x_norm2)
        
        return output

In [26]:
class SNAIL(nn.Module):
    def __init__(self, input_dim, output_dim, num_blocks, num_heads=2):
        super(SNAIL, self).__init__()
        
        # Initialize AttentionBlocks
        self.attention_blocks = nn.ModuleList([
            AttentionBlock(input_dim, output_dim, num_heads)
            for _ in range(num_blocks)
        ])
        
        # Classification layer
        self.classification_layer = nn.Linear(input_dim, 1)  # Binary classification, so output_dim is 1
        
        # Sigmoid activation for binary classification
        self.sigmoid = nn.Sigmoid()
        
        # Binary cross-entropy loss
        self.loss_function = nn.BCELoss()
        
        # Adaptive learning rate
        self.learning_rate = 0.001
        
    def forward(self, x, target):
        # Initialize optimizer
        optimizer = optim.Adam(self.parameters(), lr=self.learning_rate)
        
        for i, attention_block in enumerate(self.attention_blocks):
            # Forward pass through the AttentionBlock
            x = attention_block(x)
            
            # Calculate logits
            logits = self.classification_layer(x)
            
            # Apply sigmoid activation
            predictions = self.sigmoid(logits)
            
            # Calculate binary cross-entropy loss
            loss = self.loss_function(predictions, target)
            
            # Backpropagation and parameter update
            optimizer.zero_grad()
            loss.backward(retain_graph=True)
            optimizer.step()
            
            # Update the learning rate (optional)
            self.learning_rate = calculate_new_learning_rate(self.learning_rate, loss)
            
            print(f"Iteration {i + 1}: Loss = {loss.item()}")
        
        return predictions

In [29]:
# Extract features and labels
X = data_USG.drop(columns=['Class']).values
y = data_USG['Class'].values

# Split your data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert data to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.FloatTensor(y_train)
X_val_tensor = torch.FloatTensor(X_val)
y_val_tensor = torch.FloatTensor(y_val)

# Assuming 'y_train_tensor' and 'y_val_tensor' have shapes [batch_size]
y_train_tensor = y_train_tensor.view(-1, 1)
y_val_tensor = y_val_tensor.view(-1, 1)

# Initialize your SNAIL model
input_dim = 512  # Adjust to your input dimension
output_dim = 512  # Output dimension of AttentionBlock
num_blocks = 3  # Number of AttentionBlocks
snail_model = SNAIL(input_dim, output_dim, num_blocks)

# Define a data loader for training
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Training loop
epochs = 10  # Adjust as needed
for epoch in range(epochs):
    for batch_X, batch_y in train_loader:
        # Forward pass
        predictions = snail_model(batch_X, batch_y)

        # Calculate loss and perform backpropagation
        loss = snail_model.loss_function(predictions, batch_y.view(-1, 1))
        optimizer.zero_grad()
        loss.backward(retain_graph=True)
        optimizer.step()

    # Validate the model after each epoch (optional)
    with torch.no_grad():
        val_predictions = snail_model(X_val_tensor, y_val_tensor)
        val_loss = snail_model.loss_function(val_predictions, y_val_tensor)
    print(f"Epoch {epoch + 1}/{epochs}, Validation Loss: {val_loss.item()}")
    
# Optionally, save the trained model
torch.save(snail_model.state_dict(), 'snail_model.pth')

# Load the trained model
snail_model.load_state_dict(torch.load('snail_model.pth'))

# Convert your test data (if available) to PyTorch tensors
X_test_tensor = torch.FloatTensor(X_test)  # Replace 'X_test' with your test data

# Perform inference
with torch.no_grad():
    test_predictions = snail_model(X_test_tensor)

NameError: name 'calculate_new_learning_rate' is not defined

In [17]:
# Iterate over the DataLoader to inspect batch shapes
for batch_X, batch_y in train_loader:
    batch_size, input_dim = batch_X.shape
    print(f"Batch Shape - X: {batch_X.shape}, y: {batch_y.shape}")
    break  # Break after inspecting the first batch

# Optionally, you can also print the total number of batches
print(f"Total number of batches: {len(train_loader)}")

Batch Shape - X: torch.Size([32, 512]), y: torch.Size([32, 1])
Total number of batches: 13
